# **Model Testing**

**All Features:**

Data columns (total 29 columns):

Specific Identifiers:

- BillID      51067 non-null object


How far it made it:

- PassH       51067 non-null int64
- PassS       51067 non-null int64
- PLawDate    1393 non-null object - MAKE BOOL?
- PLawNum     51067 non-null object
- PLaw        51067 non-null int64
- Veto        51067 non-null int64

Bill Descriptive Info:

- BillType    51067 non-null object
- Title       51067 non-null object
- Cong        51067 non-null int64
- Summary     51017 non-null object - REMOVE NULL
- ImpBill     51067 non-null int64
- Chamber     51067 non-null int64 (H or S)

Info about Rep Bill Proposer:

- Cosponsr    51067 non-null float64
- IntrDate    51025 non-null object - REMOVE NULL
- Mult        51067 non-null int64
- Class       51067 non-null float64
- District    51067 non-null float64
- FrstConH    51067 non-null float64
- FrstConS    51067 non-null float64
- Gender      51067 non-null int64
- MRef        51067 non-null int64
- NameFull    51067 non-null object
- Party       51067 non-null float64
- Postal      51067 non-null object
- Majority    51067 non-null int64

Other general bill info:

- Major       51067 non-null float64
- Minor       51067 non-null float64

The URL:

- URL         51067 non-null object


List of the models to test and the parameters to tune:
  
**Models:**
- Naive Bayes
- Logistic Regression
- Random Forest
- SVM

**Evaluations:**
- Accuracy, precision, recall, f1
- Confusion Matrix
- ROC/AUC
    

**Query Data**

In [3]:
import mysql.connector 
import numpy as np
import pandas as pd
import config_final
import requests

from sodapy import Socrata
import sqlalchemy as db


In [4]:
def query(q):
    try:
        return pd.read_sql_query(q, engine)
    # Pass when no data is returned    
    except ResourceClosedError:
        pass

def query_list(col, table, distinct = True):
    elts = ['SELECT',
            'DISTINCT' if distinct else '',
            col,
            'FROM',
            table]
    query_str = ' '.join(elts)
    df = query(query_str)
    l = df.iloc[:,0].tolist()
    return l

In [5]:
engine = db.create_engine(f'mysql+mysqlconnector://{config_final.user}:{config_final.password}@{config_final.host}/con_bills')

connection = engine.connect()
metadata=db.MetaData()

In [8]:
df = query('SELECT * FROM con_bills.current_bills WHERE Cong >= 110')


In [ ]:
df = query('SELECT BillID, 
           BillType, 
           Chamber, 
           Cong, 
           Cosponsr, 
#            Title,
           IntrDate,
           Mult,
           PLaw,
           Veto,
#            Class,
           District,
           FrstConH,
           FrstConS,
           Gender,
#            NameFull,
           Party,
           Postal,
           PassH,
           PassS,
           PLawDate,
           Majority,
#            Summary 
           Topic,
           title_len         
           FROM con_bills.current_bills WHERE Cong >= 110')


In [9]:
df.head()

,BillID,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,PLaw,Title,Veto,...,PassH,PassS,PLawDate,PLawNum,ImpBill,Majority,Major,Minor,URL,Summary
0,110-HR-1,hr,0,110,205.0,2007-01-05 00:00:00,1,1,To provide for the implementation of the recom...,0,...,1,1,2007-08-03 00:00:00,Became Public Law No: 110-53.,1,1,16.0,1615.0,https://www.congress.gov/bill/110th-congress/h...,Implementing Recommendations of the 9/11 Commi...
1,110-HR-10,hr,0,110,0.0,2007-01-04 00:00:00,0,0,Reserved for Speaker.,0,...,0,0,None,0,1,1,20.0,2002.0,https://www.congress.gov/bill/110th-congress/h...,
2,110-HR-100,hr,0,110,10.0,2007-01-04 00:00:00,0,0,To amend the Higher Education Act of 1965 to p...,0,...,0,0,None,0,1,1,6.0,601.0,https://www.congress.gov/bill/110th-congress/h...,Veterans' Equity in Education Act of 2007 - Am...
3,110-HR-1000,hr,0,110,286.0,2007-02-12 00:00:00,0,0,To award a congressional gold medal to Edward ...,0,...,0,0,None,0,0,1,20.0,2006.0,https://www.congress.gov/bill/110th-congress/h...,Edward William Brooke III Congressional Gold M...
4,110-HR-1001,hr,0,110,0.0,2007-02-12 00:00:00,0,0,To amend the Haitian Hemispheric Opportunity t...,0,...,0,0,None,0,1,1,19.0,1921.0,https://www.congress.gov/bill/110th-congress/h...,Amends the Caribbean Basin Economic Recovery A...


**Final Data Cleaning**

Add:
- Len title words or anything?
- tfidf on title?

REMOVE:
- PLawNum (not necessary for analysis)
- Major/Minor (no keys)
- PLawDate (too few)
- ImpBill (almost all are)


Edit: 
- BillType: 
- Chamber: To binary
- Cong: OHE
- Postal: OHE

How to deal with: Things that happen after House passage!?

In [12]:
df.shape

(51067, 29)

In [13]:
df.isnull().sum()

BillID          0
BillType        0
Chamber         0
Cong            0
Cosponsr        0
IntrDate       42
Mult            0
PLaw            0
Title           0
Veto            0
Class           0
District        0
FrstConH        0
FrstConS        0
Gender          0
MRef            0
NameFull        0
Party           0
Postal          0
PassH           0
PassS           0
PLawDate    49674
PLawNum         0
ImpBill         0
Majority        0
Major           0
Minor           0
URL             0
Summary        50
dtype: int64

In [ ]:
#Replace Nulls in IntrDate - Figure out how to do this!
#Maybe change to just month or just year
df['IntrDate'].head()

In [ ]:
#Summary no longer necessary?
df['Summary'].fillna(' ', inplace=True)

**Final Engineering:**

In [16]:
#One hot encode Postal - SHOULD I Assign to numerical?
df['Postal'].head()

0    MS
1    CA
2    CA
3    DC
4    SC
Name: Postal, dtype: object

In [27]:
#Do I want to try to encode the name?
df['NameFull'].value_counts().count()

#one hot encode only for the top bill people
#Consider adding only top, or thinking about the particular model that is running

936

In [ ]:
#Add title length?
len(df['Title']).head()

In [ ]:
#total 325. Need to convert to groups, then OHE
df['Cosponsr'].value_counts()

# **Train and Test**

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['PassH'], axis=1)
y = df['PassH']

X_train, X_test, y_train1, y_test1 = train_test_split(X, y, test_size = .2, random_state=1)

In [ ]:
#Do i need to transform back after modeling?
encoder = preprocessing.LabelEncoder()

y_train = encoder.fit_transform(y_train1)
y_test = encoder.fit_transform(y_test1)

**One hot encoding**

All categorical variables:
- 

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

Then One-Hot-Encode:

- Encode categorical integer features using a one-hot aka one-of-K scheme.
- The input to this transformer should be a matrix of integers, denoting the values taken on by categorical (discrete) features.
- The output will be a sparse matrix where each column corresponds to one possible value of one feature.
- It is assumed that input features take on values in the range [0, n_values).
- This encoding is needed for feeding categorical data to many scikit-learn estimators, notably linear models and SVMs with the standard kernels.

# **Modeling**

Import packages:

- CountVectorizer
- TFIDF

- Naive Bayes
- Logistic Regression
- Random Forest

**Remember to look at feature importances!

In [ ]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)

# We will train our classifier with the following features:

# Numeric Features:
# - IntraDate: Datetime?


# Categorical Features:
# - District
# - Party

# - embarked: categories encoded as strings {'C', 'S', 'Q'}.
# - sex: categories encoded as strings {'female', 'male'}.
# - pclass: ordinal integers {1, 2, 3}.

#Binary Features: Will not be OHE

# We create the preprocessing pipelines for both numeric and categorical data.

numeric_features = ['']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Postal', 'Topic', 'District', 'NameFull']
categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline - laid out for each model below.

**Naive Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report


nb_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', MultinomialNB())]) #get parameters
parameters = None

nb.fit(X_train, y_train)

nb_y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(nb_y_pred, y_test))
print('f1 %s' % f1_score(nb_y_pred, y_test, average='weighted'))
print(classification_report(y_test, nb_y_pred))
print(confusion_matrix(y_test, nb_y_pred))

Use Pipeline or DFMapper!

EXAMPLE

In [ ]:

# data = data.fillna('')
 
# #Add the features of the dataframe that you want to transform and/or combine
# mapper = DataFrameMapper([
#      ('Summary', TfidfVectorizer()), #add paramaters from Topic Modeling
#      ('nb_like', None),
#      ('picture_labels', TfidfVectorizer()),
#      ('nb_share', None),
#  ])
 
# """
# Use the fit_transform method to transform the old dataframe into a new one
# that can be fed to the machine learning algorithm.
# """
 
# #sample Usage
# features = mapper.fit_transform(data)
# categories = data['INDEX New']
 
# # Split the data between train and test
# x, x_test, y, y_test = train_test_split(features,categories,test_size=0.2,train_size=0.8, random_state = 0)
 
# clf = sk.RandomForestClassifier(random_state=0)
# clf.fit(x, y)
 
# predicted = clf.predict(x_test)
 
# printreport(y_test, predicted)

Evaluate:

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

# Bring in OHE?
lr_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())]) #get parameters

# Logistic Regression Classifier
lr_classifier = LogisticRegression()

lr_classifier.fit(X_train, y_train)

lr_y_pred = lr_classifier.predict(X_test)

print('accuracy %s' % accuracy_score(lr_y_pred, y_test))
print('f1 %s' % f1_score(lr_y_pred, y_test, average='weighted'))
print(classification_report(y_test, lr_y_pred))
print(confusion_matrix(y_test, lr_y_pred))

Evaluate:
- ROC/AUC

**SGD Classifier:**

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None))])


sgd.fit(X_train, y_train)

sg_y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(sg_y_pred, y_test))
print('f1 %s' % f1_score(sg_y_pred, y_test, average='weighted'))
print(classification_report(y_test, sg_y_pred))
print(confusion_matrix(y_test, sg_y_pred))

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#add bag of trees?
#add OHE

rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(max_depth=None, max_features='auto', n_estimators=10))])


rf_pipeline.fit(X_train, y_train) 

rf_y_pred = rf_pipeline.predict(X_test)

print('accuracy %s' % accuracy_score(rf_y_pred, y_test))
print('f1 %s' % f1_score(rf_y_pred, y_test, average='weighted'))
print(classification_report(y_test, rf_y_pred))
print(confusion_matrix(y_test, rf_y_pred))

**FINAL MODEL**

**Further Evaluation**:
- Classification Report
- Confusion Matrix
- ROC/AUC

https://www.kaggle.com/selener/multi-class-text-classification-tfidf

In [ ]:
# Pretty Confusion Matrix - need to change for each model

conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(conf_mat, annot=True, cmap="Blues", fmt='d',
            xticklabels=category_id_df.Product.values, 
            yticklabels=category_id_df.Product.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title("CONFUSION MATRIX - LinearSVC\n", size=16);

# Comparing all Models:

In [ ]:
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

# 5 Cross-validation
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [ ]:

mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc